In [69]:
pip install lxml

In [70]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import requests
import lxml
import re

## Meeting Plan for Nicolas Ouellet and Benedicte Knudson:
We colaborated over Zoom and in person **two** times and used a shared Github Repository that can be found [here](https://github.com/NickOuellet/Fly-Me-To-The-Moon). <br />
We met: <br />
10/11: 5-7pm <br />
10/20: 10-12pm <br />

### Step 1

In [71]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get("https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares.html",timeout=10,headers=headers)
r.status_code

200

In [72]:
text_response = r.content #https://stackoverflow.com/questions/52389692/beautifulsoup-and-prettify-function
soup = BeautifulSoup(text_response,"html.parser")

In [73]:
table = soup.findAll("table") #find the table in the HTML

In [74]:
solar_flare_df = pd.read_html(str(table))

In [75]:
solfla_df = solar_flare_df[0]#works when we just select the first table

In [76]:
solfla_df.columns = ["rank", "x_classification", "date", "region", "start_time", "maximum_time", "end_time", "movie"] #rename columns to be more descriptive

In [77]:
solfla_df.head()

,rank,x_classification,date,region,start_time,maximum_time,end_time,movie
0,1,X28+,2003/11/04,486,19:29,19:53,20:06,MovieView archive
1,2,X20+,2001/04/02,9393,21:32,21:51,22:03,MovieView archive
2,3,X17.2+,2003/10/28,486,09:51,11:10,11:24,MovieView archive
3,4,X17+,2005/09/07,808,17:17,17:40,18:03,MovieView archive
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55,MovieView archive


### Step 2

In [78]:
solfla_df = solfla_df[["rank", "x_classification", "date", "region", "start_time", "maximum_time", "end_time"]]
solfla_df.head()

,rank,x_classification,date,region,start_time,maximum_time,end_time
0,1,X28+,2003/11/04,486,19:29,19:53,20:06
1,2,X20+,2001/04/02,9393,21:32,21:51,22:03
2,3,X17.2+,2003/10/28,486,09:51,11:10,11:24
3,4,X17+,2005/09/07,808,17:17,17:40,18:03
4,5,X14.4,2001/04/15,9415,13:19,13:50,13:55


In [79]:
import datetime

In [80]:
for index, row in solfla_df.iterrows():
    solfla_df["start_time"] = pd.to_datetime(solfla_df["start_time"])
    solfla_df["maximum_time"] = pd.to_datetime(solfla_df["maximum_time"])
    solfla_df["end_time"] = pd.to_datetime(solfla_df["end_time"])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [81]:
solfla_df.drop(columns="date",inplace=True) #drop date table

solfla_df.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,rank,x_classification,region,start_time,maximum_time,end_time
0,1,X28+,486,2021-10-22 19:29:00,2021-10-22 19:53:00,2021-10-22 20:06:00
1,2,X20+,9393,2021-10-22 21:32:00,2021-10-22 21:51:00,2021-10-22 22:03:00
2,3,X17.2+,486,2021-10-22 09:51:00,2021-10-22 11:10:00,2021-10-22 11:24:00
3,4,X17+,808,2021-10-22 17:17:00,2021-10-22 17:40:00,2021-10-22 18:03:00
4,5,X14.4,9415,2021-10-22 13:19:00,2021-10-22 13:50:00,2021-10-22 13:55:00


### Step 3

In [82]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
r = requests.get("https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html",timeout=10,headers=headers)
r.status_code

200

In [83]:
text_response = r.content #https://stackoverflow.com/questions/52389692/beautifulsoup-and-prettify-function
soup = BeautifulSoup(text_response,"html.parser")
table = soup.get_text()
split_table = table.splitlines()
split_table = split_table[15:]
split_table = split_table[0:len(split_table)-2]
split_table
nasa_df = pd.DataFrame(split_table)
nasa_df

,0
0,1997/04/01 14:00 04/01 14:15 8000 4000 S25...
1,1997/04/07 14:30 04/07 17:30 11000 1000 S28...
2,1997/05/12 05:15 05/14 16:00 12000 80 N21...
3,1997/05/21 20:20 05/21 22:00 5000 500 N05...
4,1997/09/23 21:53 09/23 22:16 6000 2000 S29...
...,...
517,2017/09/17 11:45 09/17 12:35 16000 900 S08E...
518,2017/10/18 05:48 10/18 12:40 16000 400 S06E...
519,2019/05/03 23:52 05/04 00:16 13000 2300 N12...
520,2020/11/29 13:07 11/29 15:23 14000 850 S23...
